In [1]:
# Import dependencies
from bs4 import BeautifulSoup as soup
from splinter import Browser
import pandas as pd
import time

In [2]:
url_list = ['https://www.pgatour.com/content/pgatour/stats/stat.112.y2021.html',
      'https://www.pgatour.com/content/pgatour/stats/stat.113.y2021.html']

In [3]:
# Start out dataframes as lists
df1 = []
df2 = []

dataframe = [df1, df2]

In [4]:
for index, url in enumerate(url_list):
    
    # Set the executable path and initialize the chrome browser in splinter
    executable_path = {'executable_path': 'chromedriver'}
    browser = Browser('chrome', **executable_path)
    
    # Pause 1 second
    time.sleep(1)
    
    browser.visit(url)
    browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)
    html = browser.html
    golf_soup = soup(html, 'html.parser')
    table = golf_soup.find('table', class_='table-styled')
    
    table_rows = table.find('tbody').find_all('tr')
    table_headings = table.find('thead').find_all('th')
    
    column_names = [cm.text.strip() for cm in table_headings]
    dataframe[index].append(column_names)
    
    for tr in table_rows:
        rows = [tr.text.strip() for tr in tr.find_all('td')]
        dataframe[index].append(rows)
    
    # Convert list to pandas dataframe
    dataframe[index]= pd.DataFrame(dataframe[index])
    
    # Promote headers
    header = dataframe[index].iloc[0]
    dataframe[index] = dataframe[index][1:]
    dataframe[index].columns = header
    
    # Set Index equal to PLAYER NAME
    dataframe[index] = dataframe[index].set_index('PLAYER NAME')
    
    # Quit browser
    browser.quit()

In [5]:
dataframe[1]

,RANK THIS WEEK,RANK LAST WEEK,ROUNDS,%,PAR 4 BIRDIES OR BETTER,PAR 4 HOLES
PLAYER NAME,,,,,,
Justin Thomas,1,1,45,23.50,102,434
Joaquin Niemann,2,3,55,21.68,121,558
Peter Malnati,3,2,52,21.56,119,552
Patrick Reed,4,5,39,21.43,78,364
Sam Burns,5,7,50,21.32,110,516
...,...,...,...,...,...,...
Michael Gellerman,212,,25,10.81,28,259
Mark Anderson,213,210,36,10.48,39,372
Luke Donald,214,211,32,10.29,35,340


In [6]:
df0 = dataframe[0][["PAR 3 BIRDIES OR BETTER"]]
df1 = dataframe[1][["PAR 4 BIRDIES OR BETTER"]]

In [7]:
df0.join(df1)

,PAR 3 BIRDIES OR BETTER,PAR 4 BIRDIES OR BETTER
PLAYER NAME,,
Justin Rose,26,53
Stewart Cink,41,105
Justin Thomas,34,102
Dustin Johnson,27,72
Harry Higgs,38,91
...,...,...
Ben Martin,8,42
Byeong Hun An,13,81
Jonathan Byrd,7,41
